In [1]:
import os
print(os.getcwd())  # 현재 작업 디렉토리(Current Working Directory) 출력

/home/dcrc


In [5]:
import serial
import time
import pandas as pd

# 직렬 통신 설정
port = "/dev/ttyUSB0"  # 아두이노의 직렬 포트
baudrate = 9600
arduino = serial.Serial(port, baudrate, timeout=1)

# 데이터 저장용 리스트
data = []

try:
    print("Collecting data from light sensor... Press Ctrl+C to stop.")
    
    while True:
    if arduino.in_waiting > 0:
        line = arduino.readline().decode("utf-8").strip()

        # Check if the line is empty or has the expected format
        if line and ":" in line:
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
            brightness = line.split(": ")[1]
            data.append({"Timestamp": timestamp, "Brightness (%)": brightness})
        else:
            print(f"Warning: Unexpected data format: {line}")

except KeyboardInterrupt:
    print("Data collection stopped.")

finally:
    # 데이터프레임 생성
    df = pd.DataFrame(data)
    
    # 엑셀 파일로 저장
    df.to_csv("light_sensor_data.csv", index=False, encoding="utf-8")
    print("Data saved to light_sensor_data.csv")
    
    # 포트 닫기
    arduino.close()

IndentationError: expected an indented block (2481464861.py, line 17)

# 1시간 단위로 저장되는 csv 파일 생성 (현장용)

In [1]:
import serial
import time
import pandas as pd

# 직렬 통신 설정
port = "/dev/ttyUSB0"  # 아두이노의 직렬 포트
baudrate = 9600        # 아두이노와 동일한 보드레이트
arduino = serial.Serial(port, baudrate, timeout=1)

# 데이터 저장용 리스트
data = []

# 현재 날짜 초기화
current_date = time.strftime("%Y-%m-%d")

try:
    print("Collecting data from Arduino... Press Ctrl+C to stop.")
    while True:
        if arduino.in_waiting > 0:  # 데이터가 직렬 포트에 들어왔는지 확인
            line = arduino.readline().decode("utf-8").strip()  # 데이터 읽기 및 디코딩
            timestamp = time.strftime("%Y-%m-%d %H:00:00")    # 현재 시간을 정각으로 저장
            print(f"{timestamp}, Moisture: {line}%")

            # 데이터 저장 (정각 기준으로만 저장)
            if timestamp not in [entry["Timestamp"] for entry in data]:
                data.append({"Timestamp": timestamp, "Moisture (%)": line})

            # 날짜 확인
            new_date = time.strftime("%Y-%m-%d")
            if new_date != current_date:  # 날짜가 변경되었을 경우
                # 데이터프레임 생성
                df = pd.DataFrame(data)

                # 파일명 생성 (현재 날짜 기반)
                file_name = f"sensor_data_{current_date.replace('-', '')}.csv"

                # 데이터프레임을 CSV 파일로 저장
                df.to_csv(file_name, index=False, encoding="utf-8")
                print(f"Data saved to {file_name}")

                # 저장된 데이터를 초기화하고 현재 날짜 갱신
                data = []
                current_date = new_date

except KeyboardInterrupt:
    print("Data collection stopped.")

finally:
    # 마지막으로 남은 데이터를 저장
    if data:
        df = pd.DataFrame(data)
        file_name = f"sensor_data_{current_date.replace('-', '')}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8")
        print(f"Final data saved to {file_name}")

    # 포트 닫기
    arduino.close()

2024-12-13 20:00:00, Moisture:  %
2024-12-13 20:00:00, Moisture: Starting Combined Sensor Test!%
2024-12-13 20:00:00, Moisture: Soil Moisture:0%, Light Intensity:48%, Humidity:24%, Temperature:24C%
2024-12-13 20:00:00, Moisture: Soil Moisture:0%, Light Intensity:48%, Humidity:24%, Temperature:24C%
2024-12-13 20:00:00, Moisture: Soil Moisture:0%, Light Intensity:48%, Humidity:24%, Temperature:24C%
2024-12-13 20:00:00, Moisture: Soil Moisture:0%, Light Intensity:48%, Humidity:24%, Temperature:24C%
2024-12-13 20:00:00, Moisture: Soil Moisture:0%, Light Intensity:48%, Humidity:24%, Temperature:24C%
2024-12-13 20:00:00, Moisture: Soil Moisture:0%, Light Intensity:48%, Humidity:24%, Temperature:24C%
2024-12-13 20:00:00, Moisture: Soil Moisture:0%, Light Intensity:47%, Humidity:24%, Temperature:24C%
2024-12-13 20:00:00, Moisture: Soil Moisture:0%, Light Intensity:48%, Humidity:24%, Temperature:24C%
2024-12-13 20:00:00, Moisture: Soil Moisture:0%, Light Intensity:48%, Humidity:24%, Temperature

### 제대로 된 데이터 출력

In [2]:
import serial
import time
import pandas as pd

# 직렬 통신 설정
port = "/dev/ttyUSB0"  # 아두이노의 직렬 포트
baudrate = 9600        # 아두이노와 동일한 보드레이트
arduino = serial.Serial(port, baudrate, timeout=1)

# 데이터 저장용 리스트
data = []

# 현재 날짜 초기화
current_date = time.strftime("%Y-%m-%d")

try:
    print("Collecting data from Arduino... Press Ctrl+C to stop.")
    while True:
        if arduino.in_waiting > 0:  # 데이터가 직렬 포트에 들어왔는지 확인
            line = arduino.readline().decode("utf-8").strip()  # 데이터 읽기 및 디코딩
            
            # 데이터 필터링: 센서 데이터가 포함된 라인만 처리
            if "Soil Moisture:" in line and "Light Intensity:" in line:
                timestamp = time.strftime("%Y-%m-%d %H:00:00")  # 현재 시간을 정각으로 저장
                print(f"{timestamp}, {line}")

                # 데이터 저장 (정각 기준으로만 저장)
                if timestamp not in [entry["Timestamp"] for entry in data]:
                    data.append({"Timestamp": timestamp, "Sensor Data": line})

            # 날짜 확인
            new_date = time.strftime("%Y-%m-%d")
            if new_date != current_date:  # 날짜가 변경되었을 경우
                # 데이터프레임 생성
                df = pd.DataFrame(data)

                # 파일명 생성 (현재 날짜 기반)
                file_name = f"sensor_data_{current_date.replace('-', '')}.csv"

                # 데이터프레임을 CSV 파일로 저장
                df.to_csv(file_name, index=False, encoding="utf-8")
                print(f"Data saved to {file_name}")

                # 저장된 데이터를 초기화하고 현재 날짜 갱신
                data = []
                current_date = new_date

except KeyboardInterrupt:
    print("Data collection stopped.")

finally:
    # 마지막으로 남은 데이터를 저장
    if data:
        df = pd.DataFrame(data)
        file_name = f"sensor_data_{current_date.replace('-', '')}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8")
        print(f"Final data saved to {file_name}")

    # 포트 닫기
    arduino.close()


2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:65%, Humidity:24%, Temperature:23C
2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:68%, Humidity:24%, Temperature:23C
2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:70%, Humidity:25%, Temperature:23C
2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:71%, Humidity:25%, Temperature:23C
2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:71%, Humidity:25%, Temperature:23C
2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:71%, Humidity:25%, Temperature:23C
2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:71%, Humidity:25%, Temperature:23C
2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:71%, Humidity:25%, Temperature:23C
2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:71%, Humidity:25%, Temperature:23C
2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:71%, Humidity:25%, Temperature:23C
2024-12-13 20:00:00, Soil Moisture:0%, Light Intensity:71%, Humidity:25%, Temperature:23C
2024-12-13

### 각 열로 생성

In [5]:
import serial
import time
import pandas as pd

# 직렬 통신 설정
port = "/dev/ttyUSB0"  # 아두이노의 직렬 포트
baudrate = 9600        # 아두이노와 동일한 보드레이트
arduino = serial.Serial(port, baudrate, timeout=1)

# 데이터 저장용 리스트
data = []

# 현재 날짜 초기화
current_date = time.strftime("%Y-%m-%d")

try:
    print("Collecting data from Arduino... Press Ctrl+C to stop.")
    while True:
        if arduino.in_waiting > 0:  # 데이터가 직렬 포트에 들어왔는지 확인
            line = arduino.readline().decode("utf-8").strip()  # 데이터 읽기 및 디코딩
            
            # 데이터 필터링: 센서 데이터만 처리
            if "Soil Moisture:" in line and "Light Intensity:" in line:
                timestamp = time.strftime("%Y-%m-%d %H:00:00")  # 현재 시간을 정각으로 저장
                
                # 센서 데이터 파싱
                parts = line.split(", ")
                soil_moisture = parts[0].split(":")[1].strip()  # Soil Moisture 값
                light_intensity = parts[1].split(":")[1].strip()  # Light Intensity 값
                humidity = parts[2].split(":")[1].strip()  # Humidity 값
                temperature = parts[3].split(":")[1].strip()  # Temperature 값
                
                print(f"{timestamp}, Soil Moisture: {soil_moisture}, Light Intensity: {light_intensity}, Humidity: {humidity}, Temperature: {temperature}")
                
                # 데이터 저장 (정각 기준으로만 저장)
                if timestamp not in [entry["Timestamp"] for entry in data]:
                    data.append({
                        "Timestamp": timestamp,
                        "Soil Moisture (%)": soil_moisture,
                        "Light Intensity (%)": light_intensity,
                        "Humidity (%)": humidity,
                        "Temperature (C)": temperature
                    })

            # 날짜 확인
            new_date = time.strftime("%Y-%m-%d")
            if new_date != current_date:  # 날짜가 변경되었을 경우
                # 데이터프레임 생성
                df = pd.DataFrame(data)

                # 파일명 생성 (현재 날짜 기반)
                file_name = f"sensor_data_{current_date.replace('-', '')}.csv"

                # 데이터프레임을 CSV 파일로 저장
                df.to_csv(file_name, index=False, encoding="utf-8")
                print(f"Data saved to {file_name}")

                # 저장된 데이터를 초기화하고 현재 날짜 갱신
                data = []
                current_date = new_date

except KeyboardInterrupt:
    print("Data collection stopped.")

finally:
    # 마지막으로 남은 데이터를 저장
    if data:
        df = pd.DataFrame(data)
        file_name = f"sensor_data_{current_date.replace('-', '')}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8")
        print(f"Final data saved to {file_name}")

    # 포트 닫기
    arduino.close()


2024-12-13 20:00:00, Soil Moisture: 0%, Light Intensity: 72%, Humidity: 24%, Temperature: 23C
2024-12-13 20:00:00, Soil Moisture: 0%, Light Intensity: 71%, Humidity: 24%, Temperature: 23C
2024-12-13 20:00:00, Soil Moisture: 0%, Light Intensity: 71%, Humidity: 24%, Temperature: 24C
Data collection stopped.
Final data saved to sensor_data_20241213.csv


# 1초 단위로 csv 파일 생성 (발표용)

In [9]:
import serial
import time
import pandas as pd

# 직렬 통신 설정
port = "/dev/ttyUSB0"  # 아두이노의 직렬 포트
baudrate = 9600        # 아두이노와 동일한 보드레이트
arduino = serial.Serial(port, baudrate, timeout=1)

# 데이터 저장용 리스트
data = []

# 현재 날짜 초기화
current_date = time.strftime("%Y-%m-%d")

try:
    print("Collecting data from Arduino... Press Ctrl+C to stop.")
    while True:
        if arduino.in_waiting > 0:  # 데이터가 직렬 포트에 들어왔는지 확인
            line = arduino.readline().decode("utf-8").strip()  # 데이터 읽기 및 디코딩
            
            # 데이터 필터링: 센서 데이터만 처리
            if "Soil Moisture:" in line and "Light Intensity:" in line:
                timestamp = time.strftime("%Y-%m-%d %H:%M:%S")  # 현재 시간을 초 단위로 저장
                
                # 센서 데이터 파싱
                parts = line.split(", ")
                soil_moisture = parts[0].split(":")[1].strip()  # Soil Moisture 값
                light_intensity = parts[1].split(":")[1].strip()  # Light Intensity 값
                humidity = parts[2].split(":")[1].strip()  # Humidity 값
                temperature = parts[3].split(":")[1].strip()  # Temperature 값
                
                print(f"{timestamp}, Soil Moisture: {soil_moisture}, Light Intensity: {light_intensity}, Humidity: {humidity}, Temperature: {temperature}")
                
                # 데이터 저장
                data.append({
                    "Timestamp": timestamp,
                    "Soil Moisture (%)": soil_moisture,
                    "Light Intensity (%)": light_intensity,
                    "Humidity (%)": humidity,
                    "Temperature (C)": temperature
                })

                # 데이터프레임 생성
                df = pd.DataFrame(data)

                # 파일명 생성 (현재 날짜 기반)
                file_name = f"sensor_data_{current_date.replace('-', '')}.csv"

                # 데이터프레임을 CSV 파일로 저장
                df.to_csv(file_name, index=False, encoding="utf-8")
                print(f"Data saved to {file_name}")

except KeyboardInterrupt:
    print("Data collection stopped.")

finally:
    # 마지막으로 남은 데이터를 저장
    if data:
        df = pd.DataFrame(data)
        file_name = f"sensor_data_{current_date.replace('-', '')}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8")
        print(f"Final data saved to {file_name}")

    # 포트 닫기
    arduino.close()


2024-12-13 20:30:23, Soil Moisture: 0%, Light Intensity: 64%, Humidity: 24%, Temperature: 24C
Data saved to sensor_data_20241213.csv
2024-12-13 20:30:26, Soil Moisture: 0%, Light Intensity: 67%, Humidity: 24%, Temperature: 24C
Data saved to sensor_data_20241213.csv
2024-12-13 20:30:28, Soil Moisture: 0%, Light Intensity: 69%, Humidity: 25%, Temperature: 24C
Data saved to sensor_data_20241213.csv
2024-12-13 20:30:30, Soil Moisture: 0%, Light Intensity: 70%, Humidity: 25%, Temperature: 24C
Data saved to sensor_data_20241213.csv
2024-12-13 20:30:32, Soil Moisture: 0%, Light Intensity: 53%, Humidity: 25%, Temperature: 24C
Data saved to sensor_data_20241213.csv
2024-12-13 20:30:35, Soil Moisture: 0%, Light Intensity: 44%, Humidity: 25%, Temperature: 24C
Data saved to sensor_data_20241213.csv
2024-12-13 20:30:37, Soil Moisture: 0%, Light Intensity: 40%, Humidity: 25%, Temperature: 24C
Data saved to sensor_data_20241213.csv
2024-12-13 20:30:39, Soil Moisture: 0%, Light Intensity: 39%, Humidit